In [1]:
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
import matplotlib.pyplot as plt
# from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-06-08 10:48:24.263225: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def print_data_info(data, obj):
    print(f"\n{obj} Information")
    print(f"data type: {data.dtype}")
    print(f"min: {np.min(data)}\nmax: {np.max(data)}")
    print(f"shape: {data.shape}")

In [3]:
batch_size = 10
img_height = 184
img_width = 216
img_size = (img_height, img_width)
img_shape = img_size + (3,)

images_path = "/drive0-storage/Gracia/dataset/sz"
with tf.device("CPU"):
    real_images = image_dataset_from_directory(images_path,
                                               image_size = img_size,
                                               batch_size = batch_size,
                                               seed = 123)

    for images, labels in real_images.take(1):
        for i in range(1):
            sample = images[i]
            print_data_info(sample, "Initial Real Image")

2023-06-08 10:48:26.351331: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-08 10:48:27.083096: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7910 MB memory:  -> device: 0, name: A100-SXM4-40GB MIG 2g.10gb, pci bus id: 0000:90:00.0, compute capability: 8.0


Found 4809 files belonging to 1 classes.

Initial Real Image Information
data type: <dtype: 'float32'>
min: 0.0
max: 245.0
shape: (184, 216, 3)


In [4]:
BUFFER_SIZE = 1000
AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
#     image = tf.image.resize(299,299,3)
    image = tf.image.rgb_to_grayscale(image)
    return image

with tf.device("CPU"):
    real_images = (
        real_images
        .shuffle(buffer_size = BUFFER_SIZE)
        .map(preprocess, num_parallel_calls=AUTOTUNE))
    
#     n = int(input('number of batch: '))
    n = 50
    images = real_images.take(n)

In [5]:
# Load the pre-trained InceptionV3 model
with tf.device("CPU"):
    inception_model = tf.keras.models.load_model('../inception_model.h5')

In [6]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(23*27*512, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((23, 27, 512)))
    assert model.output_shape == (None, 23, 27, 512) 

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 46, 54, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 92, 108, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 184, 216, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (4, 4), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 184, 216, 1)

    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[184, 216, 1]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

with tf.device("CPU"):
    generator = make_generator_model()
    discriminator = make_discriminator_model()

In [7]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

checkpoint_dir = './training_checkpoints'
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [8]:
from scipy.linalg import sqrtm
import numpy as np
from scipy.stats import entropy

noise_dim = 100
num_examples_to_generate = batch_size*n

def preprocess_prediction(image):
#     image = tf.image.grayscale_to_rgb(image)
#     image = tf.image.resize(images, (299,299,3))
    image = image/2 + 0.5
    image *= 255
    return image

def get_features():
    with tf.device("CPU"):
        seed = tf.random.normal([num_examples_to_generate, noise_dim])
        predictions = generator(seed, training=False)
        generated_images = preprocess_prediction(predictions)

#         print_data_info(generated_images, "Preprocessed Generated Images")
#         for image in real_images.take(1):
#             print_data_info(image, "Preprocessed Real Images")

        # calculate features from inception model prediction
        generated_features = inception_model.predict(generated_images)
        real_features = inception_model.predict(images)
    return generated_features, real_features


def calculate_fid_score(real_features, generated_features):
    # Calculate the mean of the real and generated features
    mean_real = np.mean(real_features, axis=0)
    mean_generated = np.mean(generated_features, axis=0)

    # Calculate the covariance matrices of the real and generated features
    cov_real = np.cov(real_features, rowvar=False)
    cov_generated = np.cov(generated_features, rowvar=False)

    # Calculate the squared Euclidean distance between the mean feature vectors
    distance = np.sum((mean_real - mean_generated) ** 2)

    # Calculate the square root of the product of the covariance matrices
    cov_sqrt = sqrtm(cov_real.dot(cov_generated))

    # Check and correct imaginary numbers from sqrt
    if np.iscomplexobj(cov_sqrt):
        cov_sqrt = cov_sqrt.real
    
    # Calculate the FID score
    fid_score = distance + np.trace(cov_real + cov_generated - 2 * cov_sqrt)

    return fid_score

def calculate_inception_score(generated_features):
    # Calculate the marginal distribution of class probabilities
    marginal_probs = np.mean(generated_features, axis=0)

    # Calculate the conditional distribution of class probabilities given generated images
    conditional_probs = generated_features / np.sum(generated_features, axis=1, keepdims=True)

    # Reshape the marginal_probs array to match the shape of conditional_probs
    marginal_probs_reshaped = np.tile(marginal_probs, (conditional_probs.shape[0], 1))

    # Calculate the KL-divergence between the marginal and conditional distributions
    kl_divergence = entropy(marginal_probs_reshaped.T, conditional_probs.T)

    # Calculate the Inception Score as the exponential of the mean KL-divergence
    inception_score = np.exp(np.mean(kl_divergence))

    return inception_score

def restore_specific_checkpoint(index):
    ckpt_path = f"./training_checkpoints/ckpt-{index}"
    checkpoint.restore(ckpt_path)

In [9]:
FID = []
inception_scores = []
checkpoint_idx = np.arange(10,21,1)

with tf.device("CPU"):
    for idx in checkpoint_idx:
        restore_specific_checkpoint(idx)
        generated_features, real_features = get_features()
        # calculate FID & inception scores
        inception_score = calculate_inception_score(generated_features)
        fid_score = calculate_fid_score(real_features, generated_features)
        # add to list
        inception_scores.append(inception_score)
        FID.append(fid_score)

50/50 [==============================] - 6s 110ms/step


In [10]:
import pandas as pd

df = pd.DataFrame({'epoch' : checkpoint_idx*30,
                   'checkpoint' : checkpoint_idx,
                   'FID' : FID,
                   'Inception Score':inception_scores})

df

,epoch,checkpoint,FID,Inception Score
0,300,10,0.259765,2.564888
1,330,11,0.287182,2.581723
2,360,12,0.333351,2.802733
3,390,13,0.217493,2.630947
4,420,14,0.264703,2.659037
5,450,15,0.299887,2.804519
6,480,16,0.277846,2.649082
7,510,17,0.279680,2.840089
8,540,18,0.304787,2.755237
9,570,19,0.341745,2.774896
